In [1]:
import numpy as np
import pandas as pd

In [2]:
events_df = pd.read_csv('D:\\Jupyter notebook\\events.csv')
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


看看有哪些行為類別

In [3]:
events_df.event.value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [6]:
item_df = pd.read_csv('D:\\Jupyter notebook\\item_properties_part1.csv')
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


axis=0就可以把part1跟part2接起來

In [7]:
item_df = pd.concat([item_df, pd.read_csv('D:\\Jupyter notebook\\item_properties_part2.csv')], axis=0)
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [8]:
item_df.nunique()

timestamp         18
itemid        417053
property        1104
value        1966868
dtype: int64

In [9]:
item_df = item_df.loc[item_df.itemid.isin(events_df.itemid)]
item_df.nunique()

timestamp         18
itemid        185246
property        1098
value        1000100
dtype: int64

把舊的資料去掉
 keep='last'資料若重複留最後一個

In [13]:
item_df = item_df.drop_duplicates(subset=['itemid', 'property'], keep='last')
item_df.nunique()

timestamp        18
itemid       185246
property       1098
value        932001
dtype: int64

In [14]:
item_df.loc[item_df.property=='790'].value.value_counts().to_frame()

,value
n0.000,1769
n8400.000,706
n9480.000,494
n8280.000,479
n7800.000,478
...,...
n606120.000,1
n1323240.000,1
n583440.000,1
n4459080.000,1


In [15]:
item_df.loc[item_df.property=='790'].value.apply(lambda x:x[0]!='n').sum()
#全部資料都有n

0

In [16]:
#所有商品的764屬性都是同樣的值(推薦時不需要)
item_df.loc[item_df.property=='764'].value.value_counts().to_frame()

,value
1285872,185246


In [17]:
#所有商品的159屬性都是同樣的值(推薦時不需要)
item_df.loc[item_df.property=='159'].value.value_counts().to_frame()

,value
519769,185246


In [18]:
#available 0=為販售 1=已販售
item_df.loc[item_df.property=='available'].value.value_counts().to_frame()

,value
0,139182
1,46064


In [19]:
#所有商品的364屬性變異太大所以也不好推薦
#變異大到快要等於物品id
item_df.loc[item_df.property=='364'].value.value_counts().to_frame()

,value
560583,2
221208,2
418587,2
143136,2
1276398,1
...,...
454161,1
851906,1
750502,1
622553,1


In [20]:
#很多 是常string有可能是物品名稱或是描述
#對基於內容屬性有幫助
item_df.loc[item_df.property=='888'].value.value_counts().to_frame()

,value
769062,32
138668,25
153900,25
326996,24
1177237,22
...,...
888666 934549 1083094 962642,1
1165088 1172681 424566,1
846884 106039 616467,1
572788 1192551 1076187,1


In [21]:
item_df.loc[item_df.property=='678'].value.value_counts().to_frame()

,value
820477,4582
286484,2970
212349,2745
367680,2558
1115724,2450
...,...
1052991,1
287601 234027,1
724812,1
874233 715832,1


In [22]:
item_df.loc[item_df.property=='678'].value.nunique()
#可能是產地or城市
#對基於內容推薦也有幫助

3448

In [23]:
#只留下對推薦有幫助的
item_df = item_df.loc[item_df.property.isin(['categoryid','888','678','790'])]
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
16,1435460400000,178601,790,n5400.000
40,1433646000000,152892,888,599031
42,1431831600000,125874,790,n39588.000
46,1433646000000,272201,790,n10320.000
